# Project 1 

## Group One: Covid 19 | Part One: Data Cleansing 

Team Members: Michael Bien, Lupita Lopez, Jinah Porter, Debra Wu

### Research Questions to Answer:

1. WORLD: Compare COVID deaths by GDP by Country: Does a higher GDP result in a lower number of COVID deaths?

2. US: Examine death rates by age group and determine is children under 10 die at a lower rate, adults 60+ at a higher rate

3. US: Compare cases/deaths by state

4. US: Does pre-COVID state GDP related to the cases/deaths in a given state

5. March shutdown, 4th of July and Memorial Day: compare 2 months of shutdown vs. reopening: how did reopening affect case and death rates? 

6. US: Is there a relationship between COVID cases/deaths and gender?

7. US: Is there a relationship between COVID cases/deaths and ethnicity?


In [173]:
#Import necessary libraries
import pandas as pd
import os
import requests
import numpy as np
import requests
from pprint import pprint
import json

Import World Bank data by Country; source: https://databank.worldbank.org/reports.aspx?source=2&series=NY.GDP.MKTP.CD&country=#

In [174]:
# Import world_bank_data.csv
file_path = os.path.join("..","source_data","world_bank_data.csv")
world_bank_df = pd.read_csv(file_path)
# Drop countries where the 2018 GDP is null, this is indicated by ".."
#Logic: Countries with no GDP data do not contribute to the analysis by GDP
world_bank_df = world_bank_df[world_bank_df["2018 [YR2018]"] != ".."]
world_bank_df.tail(45)

,Country Name,Country Code,2018 [YR2018]
216,Zimbabwe,ZWE,24311560500
217,Arab World,ARB,2.77E+12
218,Caribbean small states,CSS,73459161489
219,Central Europe and the Baltics,CEB,1.64E+12
220,Early-demographic dividend,EAR,1.16E+13
221,East Asia & Pacific,EAS,2.63E+13
222,East Asia & Pacific (excluding high income),EAP,1.66E+13
223,East Asia & Pacific (IDA & IBRD countries),TEA,1.66E+13
224,Euro area,EMU,1.37E+13
225,Europe & Central Asia,ECS,2.31E+13


In [175]:
# dropping rows by index number range. These rows represent aggregations of the country data and are not useful in the analysis
world_bank_df.drop(world_bank_df.loc[217:263].index, inplace=True)
world_bank_df.tail(5)

,Country Name,Country Code,2018 [YR2018]
211,Vietnam,VNM,2.45E+11
213,West Bank and Gaza,PSE,14615900000
214,"Yemen, Rep.",YEM,27591261663
215,Zambia,ZMB,27005238896
216,Zimbabwe,ZWE,24311560500


Johns Hopkins Country Data with iso: https://documenter.getpostman.com/view/10724784/SzYXWz3x?version=latest
Johns Hopkins Case Data by Country: https://documenter.getpostman.com/view/5352730/SzYbyxR5?version=latest

In [176]:
# Grab Johns Hopkins country list with 3 character country code
JH_countries = requests.get("https://covid-api.com/api/regions").json()
JH_countries_list_df = pd.DataFrame(JH_countries["data"])
JH_countries_list_df = JH_countries_list_df[["name","iso"]]
JH_countries_list_df = JH_countries_list_df.sort_values(by='name', ascending=True)
JH_countries_list_df.head()

,name,iso
37,Afghanistan,AFG
106,Albania,ALB
41,Algeria,DZA
70,Andorra,AND
178,Angola,AGO


In [258]:
# create empty columns for deaths, confirmed, recovered
JH_countries_list_df["JH Deaths"] = ""
JH_countries_list_df["JH Confirmed"] = ""
JH_countries_list_df["JH Recovered"] = ""
# create empty list to gather country lookuop errors
JH_country_no_data = []

# John Hopkins base url
base_url = "https://covid19-stats-api.herokuapp.com/api/v1/cases?country="
for index,row in JH_countries_list_df.iterrows():
    country_lookup = row["name"]
    request_url = base_url + country_lookup
    response = requests.get(request_url).json()
    try:
        confirm_JH = response["confirmed"]
        row["JH Confirmed"] = confirm_JH
        death_JH = response["deaths"]
        row["JH Deaths"] = death_JH
        recovered_JH = response["recovered"]
        row["JH Recovered"] = recovered_JH
        print(f"Processing country | {country_lookup} | {response}")
        
    except KeyError:
        print(f"<<Data not found for {country_lookup}")
        JH_country_no_data.append(country_lookup)
        pass
print("** API call complete**")

Processing country | Afghanistan | {'confirmed': 38815, 'deaths': 1426, 'recovered': 32098}
Processing country | Albania | {'confirmed': 11672, 'deaths': 340, 'recovered': 6668}
Processing country | Algeria | {'confirmed': 48734, 'deaths': 1632, 'recovered': 34385}
Processing country | Andorra | {'confirmed': 1438, 'deaths': 53, 'recovered': 945}
Processing country | Angola | {'confirmed': 3569, 'deaths': 139, 'recovered': 1332}
Processing country | Antigua and Barbuda | {'confirmed': 95, 'deaths': 3, 'recovered': 91}
Processing country | Argentina | {'confirmed': 577338, 'deaths': 11852, 'recovered': 438883}
Processing country | Armenia | {'confirmed': 46119, 'deaths': 920, 'recovered': 41941}
Processing country | Australia | {'confirmed': 26778, 'deaths': 824, 'recovered': 23650}
Processing country | Austria | {'confirmed': 34305, 'deaths': 757, 'recovered': 27354}
Processing country | Azerbaijan | {'confirmed': 38517, 'deaths': 566, 'recovered': 35998}
Processing country | Bahamas |

In [257]:
print(JH_country_no_data)

[]


In [252]:
# Drop rows with zero death using the following logic:
# 1. The list of countries and case data were obtained independently from two different Johns Hopkins apis
# 2. Instance where the Johns Hopkins country returns a null for Johns Hopkins cases indicates that the countries with null cases can be droppped as the case data is not being tracked
JH_countries_list_df = JH_countries_list_df[JH_countries_list_df["JH Deaths"] != ""]
JH_countries_list_df.shape

(188, 5)

In [253]:
# merge World Bank and Johns Hopkins dataframes on iso
#rename columns for merge
JH_countries_list_df.rename(columns={"iso":"Country Code"}, inplace=True)
WB_JH_merge_df = pd.merge(world_bank_df,JH_countries_list_df,on="Country Code",how="outer")
WB_JH_merge_df.head()

,Country Name,Country Code,2018 [YR2018],name,JH Deaths,JH Confirmed,JH Recovered
0,Afghanistan,AFG,19484384937,Afghanistan,1426,38815,32098
1,Albania,ALB,15147020535,Albania,340,11672,6668
2,Algeria,DZA,1.74E+11,Algeria,1632,48734,34385
3,American Samoa,ASM,636000000,NaN,NaN,NaN,NaN
4,Andorra,AND,3218316013,Andorra,53,1438,945


In [254]:
# capture rows where there is no JH data
JH_no_data = WB_JH_merge_df[WB_JH_merge_df["JH Deaths"].isnull()]
JH_no_data

,Country Name,Country Code,2018 [YR2018],name,JH Deaths,JH Confirmed,JH Recovered
3,American Samoa,ASM,636000000,NaN,NaN,NaN,NaN
33,Cayman Islands,CYM,5485419417,NaN,NaN,NaN,NaN
46,Curacao,CUW,3127908045,NaN,NaN,NaN,NaN
69,Greenland,GRL,3051626390,NaN,NaN,NaN,NaN
71,Guam,GUM,5920000000,NaN,NaN,NaN,NaN
78,"Hong Kong SAR, China",HKG,3.62E+11,NaN,NaN,NaN,NaN
92,Kiribati,KIR,196737896,NaN,NaN,NaN,NaN
94,Kosovo,XKX,7942961738,NaN,NaN,NaN,NaN
105,"Macao SAR, China",MAC,55084050790,NaN,NaN,NaN,NaN
112,Marshall Islands,MHL,221278000,NaN,NaN,NaN,NaN


In [255]:
# drop nulls where there is no JH data - logic:
#This analysis is of case data:
# 1.coutries without case data do not contribute to the analysis 
# 2. The dataset was joined on the iso3, which provides an exact match between datasets
WB_JH_merge_df = WB_JH_merge_df[WB_JH_merge_df["JH Deaths"].notnull()]
# Replace blank Country Names with name
WB_JH_merge_df["Country Name"].fillna(WB_JH_merge_df["name"], inplace=True)
# Delete name column from merged_df
del WB_JH_merge_df["name"]
# Fix minor Country Name value issues
WB_JH_merge_df["Country Name"]= WB_JH_merge_df["Country Name"].replace("Taiwan*", "Taiwan")
WB_JH_merge_df["Country Name"]= WB_JH_merge_df["Country Name"].replace("Korea, Rep.", "South Korea")
WB_JH_merge_df.head()

,Country Name,Country Code,2018 [YR2018],JH Deaths,JH Confirmed,JH Recovered
0,Afghanistan,AFG,19484384937,1426,38815,32098
1,Albania,ALB,15147020535,340,11672,6668
2,Algeria,DZA,1.74E+11,1632,48734,34385
4,Andorra,AND,3218316013,53,1438,945
5,Angola,AGO,1.01E+11,139,3569,1332


In [256]:
# export merged dataframe to csv
output_path = os.path.join("..","source_data","temp.csv")
WB_JH_merge_df.to_csv(output_path)

PermissionError: [Errno 13] Permission denied: '..\\source_data\\temp.csv'

In [ ]:
countries_not_on_WB = WB_JH_merge_df[WB_JH_merge_df["Country Name"].isnull()]
countries_not_on_WB.shape

In [ ]:
# Read population table from wiki
WIKI_population_df = pd.read_html("https://en.wikipedia.org/wiki/List_of_countries_and_dependencies_by_population_density")
# Return the first table in the list of tables
WIKI_population_df = WIKI_population_df[0]
# use join to rename double header of table
WIKI_population_df.columns = ["_".join(col) for col in WIKI_population_df.columns]
# remove rows that represent the double footer
WIKI_population_df = WIKI_population_df[WIKI_population_df["Rank_Rank"] != "Rank"]
# rename columns
WIKI_population_df.rename(columns={"Rank_Rank": "Rank", "Country (or dependent territory)_Country (or dependent territory)": "Country Name","Area_km2": "Area (km2)","Area_mi2": "Area (mi2)","Population_Population": "Population"}, inplace=True)
WIKI_population_df.rename(columns={"Density_pop./km2": "Population Density (km2)","Density_pop./mi2": "Population Density (mi2)"}, inplace=True)
# limit the df to the necessary columns
WIKI_population_df = WIKI_population_df[["Country Name", "Area (mi2)" , "Population", "Population Density (mi2)"]]
WIKI_population_df
# normalize WIKI_population Country Name values
WIKI_population_df["Country Name"]= WIKI_population_df["Country Name"].replace("Vatican City[note 1]", "Holy See")

In [ ]:
# merge WB_JH_merge_df with WIKI_population_df
WB_JH_WIKI_merge_df = pd.merge(WB_JH_merge_df,WIKI_population_df,on="Country Name",how="outer")
WB_JH_WIKI_merge_df.head()

In [ ]:
WB_JH_WIKI_merge_df = WB_JH_WIKI_merge_df[WB_JH_WIKI_merge_df["Country Code"].isnull()]
WB_JH_WIKI_merge_df.head(50)

In [ ]:
CDC data by gender: https://data.cdc.gov/resource/vsak-wrfu.json